In [4]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator
from keras.applications import VGG16
from keras import layers, models, optimizers
import os

# Define the path to the dataset
fer2013 = '/Users/engr/Desktop/FER2013'

# Create an ImageDataGenerator for data augmentation
datagen = ImageDataGenerator(rescale=1./255,
                             rotation_range=20,
                             width_shift_range=0.2,
                             height_shift_range=0.2,
                             shear_range=0.2,
                             zoom_range=0.2,
                             horizontal_flip=True,
                             fill_mode='nearest',
                             validation_split=0.2) 

# Specify the batch size
batch_size = 32

# Create training data generator
train_generator = datagen.flow_from_directory(
    os.path.join(fer2013, 'train'),
    target_size=(48, 48),
    batch_size=batch_size,
    class_mode='categorical',
    subset='training')

# Create validation data generator
val_generator = datagen.flow_from_directory(
    os.path.join(fer2013, 'train'),  
    target_size=(48, 48),
    batch_size=batch_size,
    class_mode='categorical',
    subset='validation')  

# Use VGG16 as a feature extractor
base_model = VGG16(weights='imagenet', include_top=False, input_shape=(48, 48, 3)) 

# Create a new model on top
model = models.Sequential()
model.add(layers.UpSampling2D((2, 2), input_shape=(48, 48, 3))) 
model.add(base_model)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(7, activation='softmax'))

# Freeze the convolutional base
base_model.trainable = False

# Compile the model
model.compile(optimizer=optimizers.Adam(lr=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model using data generators
epochs = 10

history = model.fit(
    train_generator,
    steps_per_epoch=train_generator.samples // batch_size,
    epochs=epochs,
    validation_data=val_generator,
    validation_steps=val_generator.samples // batch_size)

# Evaluate the model
val_loss, val_accuracy = model.evaluate(val_generator)
print(f'Validation accuracy: {val_accuracy * 100:.2f}%')


Found 22968 images belonging to 7 classes.
Found 5741 images belonging to 7 classes.


Epoch 1/10
717/717 [==============================] - 5132s 7s/step - loss: 1.7242 - accuracy: 0.3058 - val_loss: 1.6298 - val_accuracy: 0.3623
Epoch 2/10
 68/717 [=>............................] - ETA: 24:40 - loss: 1.6799 - accuracy: 0.3240

KeyboardInterrupt: 

The necessary libraries are imported.

The path to the FER2013 dataset is specified.

An ImageDataGenerator is created for data augmentation. This helps improve the model's generalization by applying various transformations to the training images.

The batch size is set to 32. It is the number of samples processed in one iteration during training.

Training and validation data generators are created using the flow_from_directory method. It assumes a directory structure where each class has its own subdirectory.

VGG16 is used as a feature extractor with weights pre-trained on ImageNet. The top layer is excluded, and the input shape is specified.

A sequential model is built by adding layers. An upsampling layer is used to match the input size expected by VGG16. The fully connected layers are added on top of the VGG16 base.

The weights of the convolutional base (VGG16) are frozen to prevent them from being updated during training.

The model is compiled with the Adam optimizer, categorical crossentropy loss, and accuracy as the metric.

The model is trained using the specified data generators and the number of epochs.

The model is evaluated on the validation set, and the accuracy is printed.